### Env 실험 및 영상 저장을 위한 노트북

(gym_env_test_plot 노트북 참조)

In [ ]:
import horcrux_terrain_v1
import gymnasium as gym
import numpy as np
import pandas as pd

import os
import pathlib
import time
import mediapy as media

import matplotlib.pyplot as plt

from gymnasium.utils.save_video import save_video